Ridge Regression on MNIST data

In [ ]:
import numpy as np
import scipy.linalg as sc
import mnist

class RidgeReg:

    def load_dataset(self):
        mndata = mnist.MNIST("data/")
        X_train, labels_train = map(np.array, mndata.load_training())
        X_test, labels_test = map(np.array, mndata.load_testing())
        self.X_train = X_train / 255.0
        self.X_test = X_test / 255.0

        self.Y_train = labels_train
        self.Y_test = labels_test

        # converted labels to one_hot of y in {0,1}^10
        self.labels_train = self.one_hot(labels_train)
        self.labels_test = self.one_hot(labels_test)

    # convert Y to a one_hot matrix
    def one_hot(self, Y):
        converted = np.zeros((len(Y), 10))
        for i in range(len(Y)):
            converted[i, Y[i]] = 1
        return converted

    def train(self, X, Y, reg_lambda):
        reg_matrix = reg_lambda * np.eye(X.shape[1])
        omega = sc.solve((X.T.dot(X) + reg_matrix), X.T.dot(Y))
        return omega


    def predict(self, W, X_):
        Y_p = np.zeros(len(X_))
        index = 0
        for x in X_:
            Y_p[index] = np.argmax(W.T.dot(x.T))
            index += 1
        return Y_p

    # If the results are 100% accurate, all the elements would
    # be the same, and their difference would be 0.
    # I used Y_p - Y to make a matrix where the correct predictions
    # are 0, and the wrong predictions are != 0.
    # Count of the non-zero elements is the count of wrong predictions.
    def getError(self, Y_p, Y):

        temp = Y_p - Y
        error = np.count_nonzero(temp) / len(temp)
        return error


R = RidgeReg()
R.load_dataset()

X_train = R.X_train
X_test = R.X_test
labels_train = R.labels_train
labels_test = R.labels_test

omega = R.train(X_train, labels_train, 1E-4)
Y_train_p = R.predict(omega, X_train)
Y_test_p = R.predict(omega, X_test)

Y_train = R.Y_train
Y_test = R.Y_test

Y_train_error = R.getError(Y_train_p, Y_train)
Y_test_error = R.getError(Y_test_p, Y_test)

print('Training Error: %', '%.2f' % (Y_train_error * 100))
print('Testing Error: %', '%.2f' % (Y_test_error * 100))